# The Battle of Neighborhoods - Week 1

# Coursera Capstone Report

         1)Introduction Section : Discussion of the business problem and the interested audience in this project.
         2)Data Section: Description of the data that will be used to solve the problem.
         3)Methodology section : Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes
         4)Results section : Discussion of the results.
         5)Discussion section : Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
         6)Conclusion section : Report Conclusion.

### Libraries

In [5]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!pip install geopy
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import time
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


###

## 1) Introduction

        When someone or a family is trying to find the best places to live, it's always a good idea to compare cities and if possible, to compare neighborhoods to see if it suites your taste. After all, when you go to buy a car or a house or any big ticket item, you usually try out a few models or visit a few homes before you decide. The same tactic applies to finding the best places to live. It is always advisable to do it before you start planning your move or to help narrow down your choices.When thinking about the best place to live, lots of things are considered when trying to make a comparison between cities, towns, or neighborhoods. Some of theses includes:

        How can I find a convenient and enjoyable place similar to mine now in Los Angeles? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis configurations. 

  ### Business problem

        The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the data section.

        
       

## 2) Data Section:

    Description of the data and its sources that will be used to solve the problem
    
### Description of the Data:

    The following data is required to answer the issues of the problem:
    List of Boroughs and neighborhoods of Manhattan with their geodata (latitude and longitude)
    List of Subway metro stations in Manhattan with their address location
    List of apartments for rent in Manhattan area with their addresses and price
    Preferably, a list of apartment for rent with additional information, such as price, address, area, number of beds, etc
    Clustered venues for each Manhattan neighborhood 
    Venues for subway metro stations
   
### The data will be used as follows:

    Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups
    
    Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately.
    
    Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations ,create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 miles(1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
    
    Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
    
    Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.
    
    The procesing of data will allow to answer the key questions to make a decision:
    
        what is the cost of rent (per square ft) around a mile radius from each subway metro station?
        what is the area of Manhattan with best rental pricing that meets criteria established?
        What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
        What are the venues of the two best places to live? How the prices compare?
        How venues distribute among Manhattan neighborhoods and around metro stations?
        Are there tradeoffs between size and price and location?
        Any other interesting statistical data findings of the real estate and overall data.

### Venues in S Olive Street, Los Angeles for comparison to Manhattan

In [9]:
# S Olive St , Los Angeles, California, USA
address = 'S Olive St , Los Angeles'
 
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of S olive st are {}, {}.'.format(latitude, longitude))


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of S olive st are 34.0337442, -118.2665409.


In [12]:
neighborhood_latitude=34.0337442
neighborhood_longitude=-118.2665409

In [15]:
CLIENT_ID = 'ADCJV1VMO4BD2ZIR1WWMQY5FCULQLUI1MYATA4PZLTZP5ZCW' # your Foursquare ID
CLIENT_SECRET = 'VWTSYCXR2GE5LYRZCQMMGHICDYFVTXEAHLAKCANNNIWGGBYX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ADCJV1VMO4BD2ZIR1WWMQY5FCULQLUI1MYATA4PZLTZP5ZCW
CLIENT_SECRET:VWTSYCXR2GE5LYRZCQMMGHICDYFVTXEAHLAKCANNNIWGGBYX


In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ADCJV1VMO4BD2ZIR1WWMQY5FCULQLUI1MYATA4PZLTZP5ZCW&client_secret=VWTSYCXR2GE5LYRZCQMMGHICDYFVTXEAHLAKCANNNIWGGBYX&v=20180604&ll=34.0337442,-118.2665409&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
#results

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [19]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Olde Good Things,Antique Shop,34.033974,-118.267079
1,Umami Truck,Food Truck,34.033558,-118.264034
2,(the) handbag factory,Music Venue,34.037844,-118.264778
3,SUBWAY,Sandwich Place,34.032289,-118.266522
4,Food Haus Cafè,Café,34.029887,-118.268778
5,Jack in the Box,Fast Food Restaurant,34.034064,-118.264046
6,L.A. Mart,Furniture / Home Store,34.030778,-118.266490
7,Downtown Collective,Pharmacy,34.034382,-118.264308
8,Artisanal LA,Street Art,34.030300,-118.266609
9,Sports Museum of Los Angeles,Museum,34.030572,-118.264829


### Map of Los Angeles with venues 

In [25]:
# create map of Los Angeles using by latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_la)  
    
map_la

## NOTE :

 Other sections will be described in The Battle of Neighborhoods - Week 2